# Flexible MTL design for testing different configurations

- add comments


# Set up

First, we need to clone the data. Instead of manually uploading, please clone the repository from Yipeng by running:

`!git clone -b oxpet --single-branch https://weisslab.cs.ucl.ac.uk/WEISSTeaching/datasets.git`

In [1]:
!git clone -b oxpet --single-branch https://weisslab.cs.ucl.ac.uk/WEISSTeaching/datasets.git

fatal: destination path 'datasets' already exists and is not an empty directory.


Then, upload the `library.zip` file. This is a zipped version of the entire library, for is a manual process at the moment, until we work on a better way of exporting our library.

Please not that torchvision.models.utils is not supported on the environment run here. A manual fix has been added to patch this. This should be visible when you run the code. Once uploaded, unzip the library using `unzip  library.zip`

In [2]:
!unzip -o flexible_mlt.zip 

Archive:  flexible_mlt.zip
  inflating: data/data.py            
  inflating: data/__pycache__/data.cpython-38.pyc  
  inflating: data/__pycache__/data.cpython-39.pyc  
  inflating: data/__pycache__/data.cpython-37.pyc  
  inflating: criterion/loss_functions.py  
  inflating: criterion/metric_functions.py  
  inflating: criterion/criterion.py  
  inflating: criterion/__pycache__/criterion.cpython-37.pyc  
  inflating: criterion/__pycache__/loss_functions.cpython-37.pyc  
  inflating: criterion/__pycache__/loss_functions.cpython-39.pyc  
  inflating: criterion/__pycache__/metric_functions.cpython-37.pyc  
  inflating: criterion/__pycache__/criterion.cpython-39.pyc  
  inflating: main.py                 
  inflating: __init__.py             
  inflating: utils.py                
  inflating: requirements.txt        
  inflating: train.py                
  inflating: tests/test_pt_continuous_train.py  
  inflating: tests/test_loader.py    
  inflating: tests/test_model_training_class.py  

Now we must install the relevant librries from the requirements file (this should be included in library.zip). Do this by running the following:
`!python3 -m venv env`

`!source env/bin/activate`

`!pip install -r requirements.txt`

You may get an error that the latest version of numpy could not be found. You can install this manually, although I believe it should exist by defualt in the first palce.

In [3]:
!python3 -m venv mtl_test_colab
!source mtl_test_colab/bin/activate
!which python
!pip install -r requirements.txt

Error: Command '['/content/mtl_test_colab/bin/python3', '-Im', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
/bin/bash: mtl_test_colab/bin/activate: No such file or directory
/usr/local/bin/python
  Using cached fonttools-4.28.5-py3-none-any.whl (890 kB)
  Using cached h5py-3.6.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (4.1 MB)
  Using cached imageio-2.13.5-py3-none-any.whl (3.3 MB)
  Using cached matplotlib-3.5.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
ERROR: Could not find a version that satisfies the requirement numpy==1.22.0 (from versions: 1.3.0, 1.4.1, 1.5.0, 1.5.1, 1.6.0, 1.6.1, 1.6.2, 1.7.0, 1.7.1, 1.7.2, 1.8.0, 1.8.1, 1.8.2, 1.9.0, 1.9.1, 1.9.2, 1.9.3, 1.10.0.post2, 1.10.1, 1.10.2, 1.10.4, 1.11.0, 1.11.1, 1.11.2, 1.11.3, 1.12.0, 1.12.1, 1.13.0rc1, 1.13.0rc2, 1.13.0, 1.13.1, 1.13.3, 1.14.0rc1, 1.14.0, 1.14.1, 1.14.2, 1.14.3, 1.14.4, 1.14.5, 1.14.6, 1.15.0rc1, 1.15.0rc2, 1.15.0, 1.15.1, 1.15.2, 1.15.3, 1.15

## Now... you design and run the model of your choice!

```diff
!PLEASE read before running 
```
This notebook is different than the ones I've sent previously. The major difference is that it allows you to quickly and easily design the MTL model of your choice with ease, without needing to change the backend code. It also has added support for storing loss values, validation data, testing data and also the use of custom metrics and custom losses.

Please note, it does NOT currently support saving and loading of models. That is a work in progress, but I've put it as a low priority task since it seems like our models are running quickly anyways. If there are urgent issues, then I will get it done ASAP.

Please read the instructions here as they will give you a good understanding of how this is intended to be used. Run experiments on it, and if there are any issues notify me and create them on GitHub.

In [4]:
## core libraries to load


## define the tasks that you are going to use

TASKS = ['class', 'seg', 'bb']

## Loading the data

It is good practice to always laod the data first. The dataloader has been modified to take in a path input from the user, to make it more flexible.

In [5]:
## load data libraries
from data.data import OxpetDataset, RandomBatchSampler
from torch.utils.data import BatchSampler, DataLoader

In [6]:
## get your datasets

data_path = 'datasets/data_new/' # point to the directory where the data sits
trainset = OxpetDataset(data_path + 'train', TASKS)
testset = OxpetDataset(data_path + 'test', TASKS)
valset = OxpetDataset(data_path + 'val', TASKS)

## define your dataloaders
TRAIN_BATCH_SIZE = 8
TEST_BATCH_SIZE = 8
VAL_BATCH_SIZE = 8

trainloader = DataLoader(trainset, batch_size=None,
                         sampler=BatchSampler(RandomBatchSampler(trainset, TRAIN_BATCH_SIZE), batch_size=TRAIN_BATCH_SIZE, drop_last=False))

testloader = DataLoader(testset, batch_size=None,
                         sampler=BatchSampler(RandomBatchSampler(testset, TEST_BATCH_SIZE), batch_size=TEST_BATCH_SIZE, drop_last=False))
valloader = DataLoader(valset, batch_size=None,
                         sampler=BatchSampler(RandomBatchSampler(valset, VAL_BATCH_SIZE), batch_size=VAL_BATCH_SIZE, drop_last=False))


# Defining the model

In [7]:
## imports

from models.model import resnet34_bb, resnet34_bb, resnet34_class, resnet34_class_bb, resnet34_seg, resnet34_seg_bb, resnet34_seg_class, resnet34_seg_class_bb
from torch.optim import Adam

MODEL = resnet34_seg_class_bb()
OPTIMIZER = Adam(MODEL.parameters(), lr=1e-04)

Pytorch version is: 1.10.0+cu111. torchvision.models.utils does not exist
Importing load_state_dict_from_url from torch.hub instead...


## Defining the loss, metrics

This is the newest addition to the code. There are some inbuilt losses and meteics, but there is work being done on expanding these. The most important of these are the standard combined loss, which combines the losses with an added weighting. You can also design your own losses. Make sure when doing so, that you subclass from CombinedLoss (see below).

In [18]:
## example of creating your own loss
from main import CombinedLoss # ignore the CPU/GPU messages here..
import torch
class SimpleCombinedLoss(CombinedLoss):
    def __init__(self, loss_dict, weights=None):
        super(SimpleCombinedLoss, self).__init__()  # you must inherit superclass
        self.loss_dict = loss_dict  # you must define this as such


        # feel free to add any configurations for the loss of your choice
        # in this case, we've done for a simple weighted loss.
        if weights is None:
            self.weights = torch.ones(size=(len(self.loss_dict), ))
        else:
            self.weights = weights

    def forward(self, outputs, targets):
        # while the contents of the dictionary may vary, you MUST set self.losses to a dictionary that contains your losses
        self.loss_values = {
            task: weight * loss(outputs[task], targets[task]) for (task, loss), weight in zip(self.loss_dict.items(), self.weights)
        }
        return sum(self.loss_values.values())

For our case, we are interested in 3 types of losses. A loss with no weights, a loss with weights, and a loss with weights based on a prior. So...

In [19]:
from main import RandomCombinedLoss
from criterion.loss_functions import DiceLoss
from torch.nn import CrossEntropyLoss, L1Loss

LOSSES = [CrossEntropyLoss(), DiceLoss(), L1Loss()]
LOSS_DICT = {task: loss for task, loss in zip(TASKS, LOSSES)}
# print out loss dict for sanity check
print(LOSS_DICT)

LOSS1 = SimpleCombinedLoss(LOSS_DICT)  # initialise with no weights
#LOSS2 = SimpleCombinedLoss(LOSS_DICT, weights=[0.8, 0.1, 0.1]) # give most weight to class
LOSS3 = RandomCombinedLoss(LOSS_DICT, prior='constrained_bernoulli', frequency=70)  #updates every 70 minibatches, e.g. 1 epoch

{'class': CrossEntropyLoss(), 'seg': DiceLoss(), 'bb': L1Loss()}


# We are now ready to run the model!!!

To facilitate an easy way of doing this, I've created a generalised class for running and testing. It is missing some features (e.g. saving) but i'm releasing it now for user testing.

In [ ]:
from main import RunTorchModel
from criterion.metric_functions import Accuracy

# initialise run, this is analogous to model.compile() in keras
run_instance = RunTorchModel(
    model=MODEL, optimizer=OPTIMIZER, loss=LOSS1, metrics={'class':[Accuracy()]}
)

# training params
EPOCHS=5
VERBOSE=2
TRACK_HISTORY=True # to save model hist

run_instance.train(trainloader, valloader=valloader, epochs=EPOCHS, verbose=VERBOSE, track_history=TRACK_HISTORY) # analogous to model.fit() in keras
run_instance.get_history() # training history

CUDA device not detected. Running on CPU instead.
Training model...
EPOCH 1 STARTED
---------------
class tensor(0.2798, dtype=torch.float64, grad_fn=<NllLossBackward0>)
seg tensor(0.4760, grad_fn=<RsubBackward1>)
bb tensor(117.4887, dtype=torch.float64, grad_fn=<L1LossBackward0>)
Batch 1 complete. Time taken: load(0.473), train(20.5), total(21). 
class tensor(0.2591, dtype=torch.float64, grad_fn=<NllLossBackward0>)
seg tensor(0.5373, grad_fn=<RsubBackward1>)
bb tensor(111.4976, dtype=torch.float64, grad_fn=<L1LossBackward0>)


In [ ]:
run_instance.test(testloader)